In [1]:
import sys
from collections import defaultdict
from itertools import *
import random
import numpy as np
import pdb
from scipy.sparse import *

from pyspark import SparkContext
sc=SparkContext.getOrCreate()

In [2]:
#lines = sc.parallelize([(1,1,7),(1,2,6),(1,3,7),(1,4,4),(1,5,5),(1,6,4),(2,1,6),(2,2,7),(2,4,4),(2,5,3),(2,6,4)\
 #                      ,(3,2,3),(3,3,3),(3,4,1),(3,5,1),(4,1,1),(4,2,2),(4,3,2),(4,4,3),(4,5,3),(4,6,4),(5,1,1)\
#                    ,(5,3,1),(5,4,2),(5,5,3),(5,6,3)])

def parseVectorOnUser(line):
    '''
    Parse each line of the specified data file, assuming a "|" delimiter.
    Key is user_id, converts each rating to a float.
    '''
    return line[0],(line[1],float(line[2]))

def parseVectorOnItem(line):
    '''
    Parse each line of the specified data file, assuming a "|" delimiter.
    Key is item_id, converts each rating to a float.
    '''
    return line[1],(line[0],float(line[2]))

def sampleInteractions(item_id,users_with_rating,n):
    '''
    For items with # interactions > n, replace their interaction history
    with a sample of n users_with_rating
    '''
    if len(users_with_rating) > n:
        return item_id, random.sample(users_with_rating,n)
    else:
        return item_id, users_with_rating
    
def findUserPairs(item_id,users_with_rating):
    '''
    For each item, find all user-user pairs combos. (i.e. users with the same item) 
    '''
    l = []
    for user1,user2 in permutations(users_with_rating,2):
        l.append(((user1[0],user2[0]),(user1[1],user2[1])))
    return l

def keyOnFirstUser(user_pair,item_sim_data):
    '''
    For each user-user pair, make the first user's id the key
    '''
    (user1_id,user2_id) = user_pair
    return user1_id,(user2_id,item_sim_data)

def calcSim(user_pair,rating_pairs, shrink, similarity="cosine"):
    ''' 
    For each user-user pair, return the specified similarity measure,
    along with co_raters_count.
    '''
    sum_xx, sum_xy, sum_yy, sum_x, sum_y, n = (0.0, 0.0, 0.0, 0.0, 0.0, 0)
    
    if(similarity=="cosine"):
        for rating_pair in rating_pairs:
            sum_xx += np.float(rating_pair[0]) * np.float(rating_pair[0])
            sum_yy += np.float(rating_pair[1]) * np.float(rating_pair[1])
            sum_xy += np.float(rating_pair[0]) * np.float(rating_pair[1])
            # sum_y += rt[1]
            # sum_x += rt[0]
            n += 1

        cos_sim = cosine(sum_xy,np.sqrt(sum_xx),np.sqrt(sum_yy), shrink)
        return user_pair, (cos_sim,n)
    if(similarity=="jaccard"):
        for rating_pair in rating_pairs:
            sum_xx += np.float(rating_pair[0]) * np.float(rating_pair[0])
            sum_yy += np.float(rating_pair[1]) * np.float(rating_pair[1])
            sum_xy += np.float(rating_pair[0]) * np.float(rating_pair[1])
            # sum_y += rt[1]
            # sum_x += rt[0]
            n += 1
        jac_sim = jaccard(sum_xy, sum_xx, sum_yy, shrink)
        return user_pair, (jac_sim,n)

def cosine(dot_product,rating_norm_squared,rating2_norm_squared, shrink):
    '''
    The cosine between two vectors A, B
       dotProduct(A, B) / (norm(A) * norm(B))
    '''
    numerator = dot_product
    denominator = rating_norm_squared * rating2_norm_squared + shrink

    return (numerator / (float(denominator))) if denominator else 0.0

def jaccard(dot_product, den1, den2, shrink):
    '''
    The jaccard similarity between two vectors A, B
       dotProduct(A, B) / (dotProduct(A, A^t) + dotProduct(B, B^t) - dotProduct(A, B)- 
    '''
    numerator = dot_product
    denominator = den1 + den2 - dot_product + shrink

    return (numerator / (float(denominator))) if denominator else 0.0

def nearestNeighbors(user,users_and_sims,n):
    '''
    Sort the predictions list by similarity and select the top-N neighbors
    '''
    
    users_and_sims.sort(key=lambda x: x[1][0],reverse=True)
    return user, users_and_sims[:n]

def topNRecommendations(user_id,user_sims,users_with_rating,seenDict,n, shrink):
    '''
    Calculate the top-N item recommendations for each user using the 
    weighted sums method
    '''

    # initialize dicts to store the score of each individual item,
    # since an item can exist in more than one item neighborhood
    totals = defaultdict(int)
    sim_sums = defaultdict(int)

    for (neighbor,(sim,count)) in user_sims:

        # lookup the item predictions for this neighbor
        unscored_items = users_with_rating.get(neighbor,None)

        if unscored_items:
            for (item,rating) in unscored_items:
                if item not in seenDict[user_id]:

                    # update totals and sim_sums with the rating data
                    totals[item] += sim * rating
                    sim_sums[item] += sim

    # create the normalized list of scored items 
    scored_items = [(total/(sim_sums[item]+shrink),item) for item,total in totals.items()]

    # sort the scored items in ascending order
    scored_items.sort(reverse=True)

    # take out the item score
    ranked_items = [x[1] for x in scored_items]

    return user_id,ranked_items[:n]

In [3]:
trainSet = sc.textFile("../train.csv")
trainSet = trainSet.map(lambda l: l.split(','))
trainSet = trainSet.filter(lambda line: 'userId' not in line)
trainSet = trainSet.map(lambda line: (int(line[0]), int(line[1]), int(line[2])))
meanVotePerUser = trainSet.map(lambda x: (x[0], (x[2], 1)))\
                        .reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1]))\
                        .map(lambda x: (x[0], x[1][0]/x[1][1]))
meanVotePerUserDict = meanVotePerUser.collectAsMap()
lines = trainSet.map(lambda x: (x[0], x[1], x[2] - meanVotePerUserDict[x[0]]))
lines.take(5)

[(2738, 1, -5.666666666666667),
 (4716, 1, -2.9283333333333337),
 (13298, 1, 0.9902439024390244),
 (15122, 1, -0.829787234042553),
 (11326, 2, -0.833333333333333)]

In [4]:
item_user_pairs = lines.map(parseVectorOnItem).groupByKey().cache()
print("Item: ",item_user_pairs.collect()[0][0], "- User Pairs: ", list(item_user_pairs.collect()[0][1]))

Item:  32768 - User Pairs:  [(7287, -2.8645640074211505)]


In [6]:
pairwise_users = item_user_pairs.filter(
        lambda p: len(p[1]) > 1).map(
        lambda p: findUserPairs(p[0],p[1])).flatMap(lambda x: x).groupByKey()
pairwise_users.take(5)

[((6668, 5932), <pyspark.resultiterable.ResultIterable at 0x1af088f51d0>),
 ((5289, 10741), <pyspark.resultiterable.ResultIterable at 0x1af088f54a8>),
 ((11527, 9025), <pyspark.resultiterable.ResultIterable at 0x1af088f5550>),
 ((5897, 2579), <pyspark.resultiterable.ResultIterable at 0x1af088f54e0>),
 ((14077, 2113), <pyspark.resultiterable.ResultIterable at 0x1af088f55c0>)]

In [7]:
user_sims = pairwise_users.map(lambda p: calcSim(p[0],p[1],20,'jaccard'))\
                        .map(lambda p: keyOnFirstUser(p[0],p[1])).groupByKey()\
                        .map(lambda x : (x[0], list(x[1])))\
                        .map(lambda p: nearestNeighbors(p[0],p[1],50))
user_sims.take(5)

[(2,
  [(6438, (0.0, 1)),
   (842, (0.0, 1)),
   (7040, (0.0, 1)),
   (8406, (0.0, 1)),
   (7586, (0.0, 1)),
   (9278, (0.0, 1)),
   (520, (0.0, 1)),
   (6590, (0.0, 1)),
   (8228, (0.0, 1)),
   (1710, (0.0, 1)),
   (6258, (0.0, 1)),
   (4806, (0.0, 1)),
   (2436, (0.0, 1)),
   (5321, (0.0, 1)),
   (9177, (0.0, 1)),
   (10165, (0.0, 1)),
   (1481, (0.0, 1)),
   (9195, (0.0, 1)),
   (1793, (0.0, 1)),
   (2107, (0.0, 1)),
   (11501, (0.0, 1)),
   (3809, (0.0, 1)),
   (13857, (0.0, 1)),
   (10651, (0.0, 1)),
   (4813, (0.0, 1)),
   (3169, (0.0, 1)),
   (9243, (0.0, 1))]),
 (4,
  [(2942, (0.0, 1)),
   (1726, (0.0, 1)),
   (3502, (0.0, 1)),
   (11958, (0.0, 1)),
   (8296, (0.0, 1)),
   (5682, (0.0, 1)),
   (14632, (0.0, 1)),
   (11708, (0.0, 1)),
   (3748, (0.0, 1)),
   (9996, (0.0, 1)),
   (9846, (0.0, 1)),
   (8544, (0.0, 1)),
   (13338, (0.0, 1)),
   (206, (0.0, 1)),
   (4414, (0.0, 1)),
   (12822, (0.0, 1)),
   (2696, (0.0, 1)),
   (14198, (0.0, 1)),
   (2124, (0.0, 1)),
   (1426, (0.0,

In [8]:
icm = sc.textFile("../icm.csv")
icm = icm.map(lambda l: l.split(','))\
            .filter(lambda line: line[0] != 'itemId')\
            .map(lambda x: (int(x[0]), int(x[1]), 1))
        
itemSet = trainSet.map(lambda x: (x[1], x[2]))
itemsCount = trainSet.map(lambda x: (x[1],1)).reduceByKey(lambda x,y : x + y)
itemsCount_dict = itemsCount.collectAsMap()
#-----------------------------------------
featureFreq = icm.map(lambda x: (x[1],1)).reduceByKey(lambda x, y: x + y)
featureFreqDict = featureFreq.collectAsMap()
prodCount= icm.map(lambda x: x[0]).distinct().count()
featureIdf = featureFreq.map(lambda x: (x[0],np.log10(prodCount/x[1])))
featureIdfDict = featureIdf.collectAsMap()

targetUsers = sc.textFile("../target_user.csv").filter(lambda x: "userId" not in x).map(lambda x: int(x))
targets=targetUsers.collect()

######
#TEST COMPUTING THE PREDICTION NORMALIZING BY THE N OF FEATURES AND NOT THE SQRT OF IT
norms = icm.map(lambda x: (x[0],1))\
                .reduceByKey(lambda x, y: x+y).mapValues(lambda x: np.sqrt(x))\
                .collectAsMap()

normalized = icm.map(lambda x: (x[0], x[1], x[2]/norms[x[0]]))

In [9]:
#Just consider the row of the users to predict
#IF you0re asking, only the idf depends on all the training set, in fact it is computed before reducing the trainset
trainSet=trainSet.filter(lambda x: x[0] in targets)
print(trainSet.count())
data = trainSet.map(lambda x: x[2]).collect()
rows = trainSet.map(lambda x: x[0]).collect()
cols = trainSet.map(lambda x: x[1]).collect()
data.append(0)
rows.append(15364)
cols.append(37142)
userItem=csr_matrix((data,(rows,cols)))
print("userItem shape:",userItem.shape)
data = normalized.map(lambda x: x[2]).collect()
rows = normalized.map(lambda x: x[0]).collect()
cols = normalized.map(lambda x: x[1]).collect()
data.append(0)
rows.append(37142)
cols.append(80)
itemFeature = csc_matrix((data,(rows,cols)))
print("itemFeat shape:",itemFeature.shape)

46750
userItem shape: (15365, 37143)
itemFeat shape: (37143, 19716)


In [10]:
userFeature = userItem.dot(itemFeature)
userFeature.shape

(15365, 19716)

In [11]:
data = []
rows = []
cols = []
for f in featureIdfDict.keys():
    data.append(featureIdfDict[f])
    cols.append(f)
    rows.append(f)
featureIdf = csr_matrix((data,(rows,cols)))
featureIdf.shape

(19716, 19716)

In [12]:
userProfile = userFeature.dot(featureIdf)
prediction = userProfile.dot(itemFeature.transpose())
prediction.shape

(15365, 37143)

In [13]:
numberOfRecommendations=5
#TOP POPULAR
cost=8
avgRatings=itemSet.reduceByKey(lambda x,y: x+y)
avgRatings=avgRatings.map(lambda x: (x[0],x[1]/(itemsCount_dict[x[0]]+cost)))
avgRatings.take(5)
itemOrderByPop=avgRatings.sortBy(lambda x: x[1], ascending=False)
itemPop = np.array(itemOrderByPop.map(lambda x: x[0]).collect())
seenItems= trainSet.map(lambda x: (x[0],[x[1]])).reduceByKey(lambda x,y: x + y)
seenItemsDict=seenItems.collectAsMap()
#--------------------------------------------------------------

def recommendTopPop(user_id, removeSeen=True):
    seenItems = np.array(seenItemsDict[user_id])
    recommendedList = itemPop
    if(removeSeen):
        unseen_mask = np.in1d(recommendedList, seenItems, invert=True)
        recommendedList = recommendedList[unseen_mask]       
    return recommendedList[:numberOfRecommendations]


def fillWithTopPop(recommended,user):
    TopPop=recommendTopPop(user)
    for i in range (numberOfRecommendations-len(recommended)):
        recommended.append(TopPop[i])
    return recommended


def getRecommended(user):
    recommended = []
    itemsPred = prediction.getrow(user).toarray()[0]

    for i in range(0,len(itemsPred)):
        if(itemsPred[i]!=0):
                if i not in seenItemsDict[user]:
                    recommended.append((i, itemsPred[i]))
    recommended.sort(key = lambda x: -x[1])
    recommended=recommended[:numberOfRecommendations]
    recommendedItems = list(map(lambda x: x[0], recommended))
    if(len(recommendedItems)<numberOfRecommendations):
        recommendedItems=fillWithTopPop(recommendedItems, user)
    return recommendedItems    


In [14]:
userSimsForTarget= user_sims.filter(lambda x: x[0] in targets)
userSimsForTarget.count()

l1=userSimsForTarget.map(lambda x: x[0]).collect()
l2=userSimsForTarget.filter(lambda x: x[1][0][1][0]==0.0).map(lambda x: x[0]).collect()
l3=[i for i in targets if i not in l1]
userForContent=l3+l2
userForCollaborative = userSimsForTarget.filter(lambda x: x[0] not in userForContent)
userForCollaborative.count()

2964

In [15]:
user_item_hist = lines.map(parseVectorOnUser).groupByKey().collect()
ui_dict = {}
for (user,items) in user_item_hist: 
    ui_dict[user] = items

uib = sc.broadcast(ui_dict)
seenItemsDict = lines.map(lambda x: (x[0], [x[1]])).reduceByKey(lambda x,y: x+y).collectAsMap()
'''
Calculate the top-N item recommendations for each user
    user_id -> [item1,item2,item3,...]
'''


user_item_recs_collaborative = userForCollaborative.map(lambda p: topNRecommendations(p[0],p[1],uib.value,seenItemsDict,5,7)).collect()
user_item_recs_content = [] 

for us in userForContent:
    user_item_recs_content.append((us, getRecommended(us)))

userRecsFinal = user_item_recs_collaborative + user_item_recs_content
userRecsFinal.sort(key = lambda x: x[0])
userRecsFinal

[(4, [32578, 35061, 98, 30408, 10129]),
 (5, [269, 28083, 35738, 35158, 25920]),
 (8, [29915, 28966, 25363, 18016, 28109]),
 (9, [29915, 28109, 35381, 28059, 25363]),
 (13, [20620, 34351, 10701, 35300, 14709]),
 (18, [3431, 18615, 7342, 7759, 23240]),
 (19, [6627, 12354, 3431, 17389, 6461]),
 (23, [4735, 33562, 22389, 24488, 6821]),
 (26, [13069, 23833, 24256, 28592, 27139]),
 (29, [24217, 21284, 1041, 31496, 19215]),
 (31, [17126, 19798, 10963, 14820, 26723]),
 (32, [6461, 33173, 9735, 10701, 17152]),
 (33, [28346, 6539, 10620, 26098, 28738]),
 (35, [23282, 12866, 353, 34517, 31136]),
 (37, [30453, 8099, 12873, 20620, 25020]),
 (51, [31992, 3841, 23282, 24786, 12669]),
 (52, [36248, 36518, 8669, 36872, 17389]),
 (53, [23282, 6461, 4708, 18546, 3431]),
 (54, [19938, 25196, 16766, 12697, 10701]),
 (61, [15221, 5671, 24236, 32128, 30284]),
 (64, [28701, 16766, 23081, 15743, 22029]),
 (69, [7801, 10834, 33997, 30091, 10546]),
 (76, [12866, 16125, 24834, 15280, 27559]),
 (78, [35300, 6612,

In [17]:
f=open("predictionsJac20ShrinkUB.csv",'w')
i=0
f.write("userId,RecommendedItemIds\n")
for (user, prods) in userRecsFinal:
    f.write(str(user)+',')
    for prod in prods:
        f.write(str(prod)+' ')
    f.write('\n')
    i=i+1
    print(i,"of", len(userRecsFinal), "written")
f.close()

1 of 4196 written
2 of 4196 written
3 of 4196 written
4 of 4196 written
5 of 4196 written
6 of 4196 written
7 of 4196 written
8 of 4196 written
9 of 4196 written
10 of 4196 written
11 of 4196 written
12 of 4196 written
13 of 4196 written
14 of 4196 written
15 of 4196 written
16 of 4196 written
17 of 4196 written
18 of 4196 written
19 of 4196 written
20 of 4196 written
21 of 4196 written
22 of 4196 written
23 of 4196 written
24 of 4196 written
25 of 4196 written
26 of 4196 written
27 of 4196 written
28 of 4196 written
29 of 4196 written
30 of 4196 written
31 of 4196 written
32 of 4196 written
33 of 4196 written
34 of 4196 written
35 of 4196 written
36 of 4196 written
37 of 4196 written
38 of 4196 written
39 of 4196 written
40 of 4196 written
41 of 4196 written
42 of 4196 written
43 of 4196 written
44 of 4196 written
45 of 4196 written
46 of 4196 written
47 of 4196 written
48 of 4196 written
49 of 4196 written
50 of 4196 written
51 of 4196 written
52 of 4196 written
53 of 4196 written
54

894 of 4196 written
895 of 4196 written
896 of 4196 written
897 of 4196 written
898 of 4196 written
899 of 4196 written
900 of 4196 written
901 of 4196 written
902 of 4196 written
903 of 4196 written
904 of 4196 written
905 of 4196 written
906 of 4196 written
907 of 4196 written
908 of 4196 written
909 of 4196 written
910 of 4196 written
911 of 4196 written
912 of 4196 written
913 of 4196 written
914 of 4196 written
915 of 4196 written
916 of 4196 written
917 of 4196 written
918 of 4196 written
919 of 4196 written
920 of 4196 written
921 of 4196 written
922 of 4196 written
923 of 4196 written
924 of 4196 written
925 of 4196 written
926 of 4196 written
927 of 4196 written
928 of 4196 written
929 of 4196 written
930 of 4196 written
931 of 4196 written
932 of 4196 written
933 of 4196 written
934 of 4196 written
935 of 4196 written
936 of 4196 written
937 of 4196 written
938 of 4196 written
939 of 4196 written
940 of 4196 written
941 of 4196 written
942 of 4196 written
943 of 4196 written


1357 of 4196 written
1358 of 4196 written
1359 of 4196 written
1360 of 4196 written
1361 of 4196 written
1362 of 4196 written
1363 of 4196 written
1364 of 4196 written
1365 of 4196 written
1366 of 4196 written
1367 of 4196 written
1368 of 4196 written
1369 of 4196 written
1370 of 4196 written
1371 of 4196 written
1372 of 4196 written
1373 of 4196 written
1374 of 4196 written
1375 of 4196 written
1376 of 4196 written
1377 of 4196 written
1378 of 4196 written
1379 of 4196 written
1380 of 4196 written
1381 of 4196 written
1382 of 4196 written
1383 of 4196 written
1384 of 4196 written
1385 of 4196 written
1386 of 4196 written
1387 of 4196 written
1388 of 4196 written
1389 of 4196 written
1390 of 4196 written
1391 of 4196 written
1392 of 4196 written
1393 of 4196 written
1394 of 4196 written
1395 of 4196 written
1396 of 4196 written
1397 of 4196 written
1398 of 4196 written
1399 of 4196 written
1400 of 4196 written
1401 of 4196 written
1402 of 4196 written
1403 of 4196 written
1404 of 4196 

1893 of 4196 written
1894 of 4196 written
1895 of 4196 written
1896 of 4196 written
1897 of 4196 written
1898 of 4196 written
1899 of 4196 written
1900 of 4196 written
1901 of 4196 written
1902 of 4196 written
1903 of 4196 written
1904 of 4196 written
1905 of 4196 written
1906 of 4196 written
1907 of 4196 written
1908 of 4196 written
1909 of 4196 written
1910 of 4196 written
1911 of 4196 written
1912 of 4196 written
1913 of 4196 written
1914 of 4196 written
1915 of 4196 written
1916 of 4196 written
1917 of 4196 written
1918 of 4196 written
1919 of 4196 written
1920 of 4196 written
1921 of 4196 written
1922 of 4196 written
1923 of 4196 written
1924 of 4196 written
1925 of 4196 written
1926 of 4196 written
1927 of 4196 written
1928 of 4196 written
1929 of 4196 written
1930 of 4196 written
1931 of 4196 written
1932 of 4196 written
1933 of 4196 written
1934 of 4196 written
1935 of 4196 written
1936 of 4196 written
1937 of 4196 written
1938 of 4196 written
1939 of 4196 written
1940 of 4196 

2416 of 4196 written
2417 of 4196 written
2418 of 4196 written
2419 of 4196 written
2420 of 4196 written
2421 of 4196 written
2422 of 4196 written
2423 of 4196 written
2424 of 4196 written
2425 of 4196 written
2426 of 4196 written
2427 of 4196 written
2428 of 4196 written
2429 of 4196 written
2430 of 4196 written
2431 of 4196 written
2432 of 4196 written
2433 of 4196 written
2434 of 4196 written
2435 of 4196 written
2436 of 4196 written
2437 of 4196 written
2438 of 4196 written
2439 of 4196 written
2440 of 4196 written
2441 of 4196 written
2442 of 4196 written
2443 of 4196 written
2444 of 4196 written
2445 of 4196 written
2446 of 4196 written
2447 of 4196 written
2448 of 4196 written
2449 of 4196 written
2450 of 4196 written
2451 of 4196 written
2452 of 4196 written
2453 of 4196 written
2454 of 4196 written
2455 of 4196 written
2456 of 4196 written
2457 of 4196 written
2458 of 4196 written
2459 of 4196 written
2460 of 4196 written
2461 of 4196 written
2462 of 4196 written
2463 of 4196 

2997 of 4196 written
2998 of 4196 written
2999 of 4196 written
3000 of 4196 written
3001 of 4196 written
3002 of 4196 written
3003 of 4196 written
3004 of 4196 written
3005 of 4196 written
3006 of 4196 written
3007 of 4196 written
3008 of 4196 written
3009 of 4196 written
3010 of 4196 written
3011 of 4196 written
3012 of 4196 written
3013 of 4196 written
3014 of 4196 written
3015 of 4196 written
3016 of 4196 written
3017 of 4196 written
3018 of 4196 written
3019 of 4196 written
3020 of 4196 written
3021 of 4196 written
3022 of 4196 written
3023 of 4196 written
3024 of 4196 written
3025 of 4196 written
3026 of 4196 written
3027 of 4196 written
3028 of 4196 written
3029 of 4196 written
3030 of 4196 written
3031 of 4196 written
3032 of 4196 written
3033 of 4196 written
3034 of 4196 written
3035 of 4196 written
3036 of 4196 written
3037 of 4196 written
3038 of 4196 written
3039 of 4196 written
3040 of 4196 written
3041 of 4196 written
3042 of 4196 written
3043 of 4196 written
3044 of 4196 

3511 of 4196 written
3512 of 4196 written
3513 of 4196 written
3514 of 4196 written
3515 of 4196 written
3516 of 4196 written
3517 of 4196 written
3518 of 4196 written
3519 of 4196 written
3520 of 4196 written
3521 of 4196 written
3522 of 4196 written
3523 of 4196 written
3524 of 4196 written
3525 of 4196 written
3526 of 4196 written
3527 of 4196 written
3528 of 4196 written
3529 of 4196 written
3530 of 4196 written
3531 of 4196 written
3532 of 4196 written
3533 of 4196 written
3534 of 4196 written
3535 of 4196 written
3536 of 4196 written
3537 of 4196 written
3538 of 4196 written
3539 of 4196 written
3540 of 4196 written
3541 of 4196 written
3542 of 4196 written
3543 of 4196 written
3544 of 4196 written
3545 of 4196 written
3546 of 4196 written
3547 of 4196 written
3548 of 4196 written
3549 of 4196 written
3550 of 4196 written
3551 of 4196 written
3552 of 4196 written
3553 of 4196 written
3554 of 4196 written
3555 of 4196 written
3556 of 4196 written
3557 of 4196 written
3558 of 4196 

4018 of 4196 written
4019 of 4196 written
4020 of 4196 written
4021 of 4196 written
4022 of 4196 written
4023 of 4196 written
4024 of 4196 written
4025 of 4196 written
4026 of 4196 written
4027 of 4196 written
4028 of 4196 written
4029 of 4196 written
4030 of 4196 written
4031 of 4196 written
4032 of 4196 written
4033 of 4196 written
4034 of 4196 written
4035 of 4196 written
4036 of 4196 written
4037 of 4196 written
4038 of 4196 written
4039 of 4196 written
4040 of 4196 written
4041 of 4196 written
4042 of 4196 written
4043 of 4196 written
4044 of 4196 written
4045 of 4196 written
4046 of 4196 written
4047 of 4196 written
4048 of 4196 written
4049 of 4196 written
4050 of 4196 written
4051 of 4196 written
4052 of 4196 written
4053 of 4196 written
4054 of 4196 written
4055 of 4196 written
4056 of 4196 written
4057 of 4196 written
4058 of 4196 written
4059 of 4196 written
4060 of 4196 written
4061 of 4196 written
4062 of 4196 written
4063 of 4196 written
4064 of 4196 written
4065 of 4196 